In [64]:
using Expectations, ForwardDiff, QuadGK, LinearAlgebra, FastGaussQuadrature

## Trapezoidal Rule (uniform and non-uniform)
$$ \int_{x_{\min}}^{x_{\max}} f(x) dx \approx \sum_{i=1}^N f(x_i) w_i = f(\vec{x}) \cdot w $$

In [32]:
function trapz_integral(f, nodes::AbstractRange)
    M = length(nodes)
    Δ = step(nodes)
    total = zero(eltype(nodes))
    for (i, x) in enumerate(nodes)
        weight = ((i == 1) || (i == M)) ?  Δ/2 : Δ   # ternary operation, Condition ? if true : if false
        total += weight * f(x)
    end
    return total
end
f(x) = x^2
x_min = 0.0
x_max = 1.0
nodes = x_min:0.01:x_max
@show quadgk(f, x_min, x_max)[1]
@show trapz_integral(f, nodes);

(quadgk(f, x_min, x_max))[1] = 0.3333333333333333
trapz_integral(f, nodes) = 0.33335000000000015


In [61]:
function trapz_weights(nodes::AbstractRange)
    M = length(nodes)
    Δ = step(nodes)
    return [Δ/2; Δ*ones(M-2); Δ/2]
end
w = trapz_weights(nodes)
@show w ⋅ f.(nodes);

w ⋅ f.(nodes) = 0.33335000000000004


In [63]:
function trapz_weights(nodes::AbstractArray)  # Note multiple dispatch
    M = length(nodes)
    Δ = diff(nodes)
    prepend!(Δ, NaN) # To keep the indexing straight. Now, Δ[2] = Δ_2 = z_2 - z_1. And NaN will throw an error if we try to use it.
    interiorWeights = [(Δ[i] + Δ[i+1])/2 for i = 2:M-1]
    return [Δ[2]/2; interiorWeights; Δ[M]/2]
end
nodes_array = Array(nodes)
w = trapz_weights(nodes_array)
@show w ⋅ f.(nodes_array);

w ⋅ f.(nodes_array) = 0.33335


## Example: Legendre Quadrature
See https://en.wikipedia.org/wiki/Gaussian_quadrature#Gauss%E2%80%93Legendre_quadrature
$$ \int_{-1}^1 f(x) dx \approx \sum_{i=1}^N f(x_i) w_i = f(\vec{x}) \cdot w $$


In [76]:
N = 10
nodes, weights = gausslegendre(N)  # can adjust to change range
@show quadgk(f, -1.0, 1.0)[1]
dot(weights, f.(nodes)) # note few evaluations

(quadgk(f, -1.0, 1.0))[1] = 0.6666666666666667


0.666666666666667